# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import tensorflow as tf
import pandas as pd
import pickle
warnings.filterwarnings('ignore')

2024-06-20 00:09:36.668100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 00:09:37.465500: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose

# 1. Load Model

In [3]:
# path to the model
dir_model = "../model/xyz_LSTM_1/"

loaded_model = tf.saved_model.load(dir_model)
inferer = loaded_model.signatures["serving_default"]
inferer??

2024-06-20 00:09:45.445560: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 00:09:45.483824: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 00:09:45.483891: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 00:09:45.487663: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 00:09:45.487742: I external/local_xla/xla/stream_executor

Signature:      inferer(*args, **kwargs)
Type:           _WrapperFunction
String form:   
ConcreteFunction Input Parameters:
  inputs (KEYWORD_ONLY): TensorSpec(shape=(None, 40, 99), dtype=tf.float32, name='inputs')
Output Type:
  Dict[['output_0', TensorSpec(shape=(None, 2), dtype=tf.float32, name='output_0')]]
Captures:
  140678850182864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850182512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850182336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850182160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850181984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850181808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850181632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850181456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850181280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140678850181104: TensorSpec(shape=(), dtype=

# 2. Setup OpenCV and MediaPipe to Predict

In [4]:
import time

In [6]:
dir_video_test = ('../../data/momentum/good4.mp4')
cap = cv2.VideoCapture(dir_video_test)
current_stage = ''

i_momentum = 0
i_non_momentum = 0

sequences = []
#initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # stream the video
    while cap.isOpened():
        ret, frame = cap.read()

        # recolor feed
        image = cv2.resize(frame, (480, 640), interpolation= cv2.INTER_LINEAR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # make detection
        results = pose.process(image)
        
        # recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # try to predict
        try:
            # model xyz
            row = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten()
            sequences.append(row)
            sequences  = sequences[-40:]

            if len(sequences) == 40:
                momentum_class = np.argmax(inferer(inputs=(np.expand_dims(sequences, axis=0)))['output_0'].numpy())
                momentum_prob = np.max(inferer(inputs=(np.expand_dims(sequences, axis=0)))['output_0'].numpy())

            # # model xyz_hand
            # # row = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # # X = pd.DataFrame([row])
            # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # # model xy
            # # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[]]).flatten()
            # # X = pd.DataFrame([row])
            # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # # model xy_hand
            # # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # # X = pd.DataFrame([row])
            # # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

                print(momentum_class, momentum_prob)

                if momentum_class == 0 and momentum_prob >= 0.9 :
                    current_stage = 'Nice!'
                    i_non_momentum+=1
                elif momentum_class == 1 and momentum_prob >= 0.9 :
                    current_stage = 'Use momentum!'
                    i_momentum+=1

                # get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
    
                #Display class
                cv2.putText(image, 'Class', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
                #Display Prob
                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(momentum_prob, 2)), (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        
        except Exception as e:
            print(e)
            
        # Stream vid resultq
        cv2.imshow("Raw Cam Feed", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

W0000 00:00:1718817036.754426    9039 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718817036.775602    9049 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0xae099e0) is not the object's thread (0xb8dfb50).
Cannot move to target thread (0xae099e0)

QObject::moveToThread: Current thread (0xae099e0) is not the object's thread (0xb8dfb50).
Cannot move to target thread (0xae099e0)

QObject::moveToThread: Current thread (0xae099e0) is not the object's thread (0xb8dfb50).
Cannot move to target thread (0xae099e0)

QObject::moveToThread: Current thread (0xae099e0) is not the object's thread (0xb8dfb50).
Cannot move to target thread (0xae099e0)

QObject::moveToThread: Current thread (0xae099e0) is not the object's thread (0xb8dfb50).
Cannot move to tar

0 0.9982401
0 0.99810153
0 0.9979361
0 0.9977719
0 0.99762976
0 0.9974974
0 0.9973719
0 0.9972645
0 0.997186
0 0.9971233
0 0.9970726
0 0.99703777
0 0.997013
0 0.99699557
0 0.9970016
0 0.99702066
0 0.99703825
0 0.99706036
0 0.9971259
0 0.99721485
0 0.9973259
0 0.99743193
0 0.99753153
0 0.9976279
0 0.99772054
0 0.9978067
0 0.9978867
0 0.99796176
0 0.9980325
0 0.9980957
0 0.9981269
0 0.9981548
0 0.99819714


In [49]:
cap.release()
cv2.destroyAllWindows()